<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Select-events" data-toc-modified-id="Select-events-0.1">Select events</a></span><ul class="toc-item"><li><span><a href="#Save-selected-events-to-file" data-toc-modified-id="Save-selected-events-to-file-0.1.1">Save selected events to file</a></span></li></ul></li><li><span><a href="#Align-clusters" data-toc-modified-id="Align-clusters-0.2">Align clusters</a></span></li><li><span><a href="#Add-tracks-and-vertex-to-events" data-toc-modified-id="Add-tracks-and-vertex-to-events-0.3">Add tracks and vertex to events</a></span><ul class="toc-item"><li><span><a href="#Save-tracked-events-to-file" data-toc-modified-id="Save-tracked-events-to-file-0.3.1">Save tracked events to file</a></span></li><li><span><a href="#Opening-angle" data-toc-modified-id="Opening-angle-0.3.2">Opening angle</a></span></li><li><span><a href="#DCA-between-two-tracks" data-toc-modified-id="DCA-between-two-tracks-0.3.3">DCA between two tracks</a></span></li><li><span><a href="#DCA-between-vertex-and-[0,0,0]" data-toc-modified-id="DCA-between-vertex-and-[0,0,0]-0.3.4">DCA between vertex and [0,0,0]</a></span></li></ul></li></ul></li><li><span><a href="#Very-clean-events" data-toc-modified-id="Very-clean-events-1">Very clean events</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Vertex-positions" data-toc-modified-id="Vertex-positions-1.0.1">Vertex positions</a></span></li><li><span><a href="#Position-shift-between-layers" data-toc-modified-id="Position-shift-between-layers-1.0.2">Position shift between layers</a></span></li><li><span><a href="#Correlation-between-ALPIDEs-after-all-selections" data-toc-modified-id="Correlation-between-ALPIDEs-after-all-selections-1.0.3">Correlation between ALPIDEs after all selections</a></span></li></ul></li></ul></li></ul></div>

In [1]:
import math
import matplotlib.pyplot as plt
import ROOT
import copy
import numpy as np
from uits3_krakow22.src.Event import Event
from uits3_krakow22.src.Cluster import Cluster
from uits3_krakow22.src.Track import Track
from uits3_krakow22.src.Vertex import Vertex
from uits3_krakow22.src.Utils import *

from multiprocessing import Process
from multiprocessing import Pool
import time
import math

from ipywidgets import IntProgress
import matplotlib.pyplot as plt
from IPython.display import display,HTML,clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

Welcome to JupyROOT 6.24/06


### Select events (and prealign azimuth)

In [2]:
#first weekend files

#path1 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run447162516_221106162754.pkl" #200 MeV,14.9Gb, threshold not mentioned Run 57
#path2 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run445203522_221104212842.pkl" #120 MeV, 2.3Gb, threshold not mentioned Run 031, may be junk
#path3 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run446172943_221105174846.pkl" #80 MeV,  2.2Gb, threshold not mentioned Run 38


#second weekend (cleaner to use 200MeV)
path = "/home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/run456195948_221112200957.pkl" #200MeV 100e
#path = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456103757_221112104316.pkl"  #120MeV 300e 
#path = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456114514_221112120005.pkl" #80MeV 100e 


#run456114514_221112120005.pkl"
  #uits3_krakow22/output/output.pkl" #yzAlign_run456114514_221112120005.pkl" #"/home/cehrich/Software/uits3_krakow22/run457063715_221113063928_clusters.pkl" #"/home/cehrich/Software/uits3_krakow22/run456195948_221112200957.pkl" 
#first file is 80MeV, 0.6 GB, second and third are 200 MeV at 3GB and 12GB respectively (pkl size)
nEvents = -1
selectedEvents = []
if nEvents > 0 :
    f = IntProgress(min=0, max=nEvents, description="Processing:")
    display(f)

shift = -1  #azimuth shift -1° but not doing that this time
needsFlip=True

for i,event in enumerate(readEvents(path, nEvents=nEvents)):
    #print("test")
    if nEvents > 0 and i>0 and i%1e4 == 0: f.value += 1e4
    eventStatus = True
    event.clusters = [cluster.flipYaxis() for cluster in event.clusters]
    #event.clusters = [cluster.shiftXaxis("ALPIDE_1",shift, "angle") for cluster in event.clusters if 23 > cluster.size > 3]
        
    for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]:
        clusters = event.selectDetector(alpide)
        if len(clusters) != 1:
            eventStatus = False
            break
    if eventStatus : selectedEvents.append(event)
        
if 'f' in locals(): 
    f.bar_style = "success"
    print("Found events:",len(selectedEvents),", Fraction:",len(selectedEvents)*100/nEvents,"%")

Going to process all events.


#### Save selected events to file

In [3]:
path_selected = "/home/cehrich/Software/uits3_krakow22/selectedEvents_brute.pkl"
save2pickle(selectedEvents,path_selected)

### Add tracks and vertex to events

In [4]:
#path_selected = "/home/cehrich/Software/uits3_krakow22/selectedEvents.pkl" #this was defined already above
events= []
for event in readEvents("/home/cehrich/Software/uits3_krakow22/selectedEvents_brute.pkl", nEvents=-1):
    TrackLeft = Track()
    TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_4"]])
    TrackRight = Track()
    TrackRight.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
    vertex = Vertex()
    vertex.fromTracks([TrackLeft,TrackRight])
    event.addTrack(TrackLeft)
    event.addTrack(TrackRight)
    event.vertex = vertex
    events.append(event)

Going to process all events.


#### Save tracked events to file

In [5]:
path_tracked = "/home/cehrich/Software/uits3_krakow22/trackedEvents_brute.pkl"
save2pickle(events,path_tracked)

## Very clean events

In [6]:
verySelectedEvents = []
firstEvents = 0
for event in loadEvents("/home/cehrich/Software/uits3_krakow22/trackedEvents_brute.pkl"): # events: 
    if 85 < event.vertex.openingAngle*360/(2*math.pi) < 90 and abs(event.vertex.dca2origin - 18.75) < 2 and event.vertex.dca < 2 and all([7<cluster.size<19 for cluster in event.clusters]): #2sigma of 80MeV 100e target sizes
        if(all([(-8 < cluster.localPos[0] < 8 and -3 < cluster.localPos[1] < 3) for cluster in event.clusters])): #cuts out multiple scattering with plastic window
            verySelectedEvents.append(event)
            if False: #(firstEvents <= 1):
                print("\nEvent ", firstEvents)#,"\n")
                for cluster in event.clusters:
                    print("Detector: ", cluster.detector, "Global position: ", cluster.globalPos, "Local position: ", cluster.localPos, "size:", cluster.size)#, "\n")
                firstEvents+=1


In [7]:
print(len(verySelectedEvents))
path_clean = "/home/cehrich/Software/uits3_krakow22/cleanEvents_brute.pkl"
save2pickle(verySelectedEvents,path_clean)

658


# Scan alignment

In [8]:
verySelectedEvents = loadEvents(path_clean)
len(verySelectedEvents)

658

In [9]:
import copy
%jsroot on
hVertexXY = ROOT.TH2F("VertexXY","VertexXY",200,-7,7,200,-7,7)
hVertexXY.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXY.GetYaxis().SetTitle("Vertex Y position [mm]")
hVertexXZ = ROOT.TH2F("VertexXZ","VertexXZ",200,-5,5,200,-23,-15)
hVertexXZ.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXZ.GetYaxis().SetTitle("Vertex Z position [mm]")
hVertexYZ = ROOT.TH2F("VertexYZ","VertexYZ",200,-7,7,200,-26,-12)
hVertexYZ.GetXaxis().SetTitle("Vertex Y position [mm]")
hVertexYZ.GetYaxis().SetTitle("Vertex Z position [mm]")
myVerySelectedEvents=copy.deepcopy(verySelectedEvents)
for event in myVerySelectedEvents:
    hVertexXY.Fill(event.vertex.point[0],event.vertex.point[1])
    hVertexXZ.Fill(event.vertex.point[0],event.vertex.point[2])
    hVertexYZ.Fill(event.vertex.point[1],event.vertex.point[2])
canvas = ROOT.TCanvas("","",1200,400)
canvas.Divide(3)
canvas.cd(1)
ROOT.gPad.SetGrid(2)
hVertexXY.Draw("COLZ")
canvas.cd(2)
ROOT.gPad.SetGrid(2)
hVertexXZ.Draw("COLZ")
canvas.cd(3)
ROOT.gPad.SetGrid(2)
hVertexYZ.Draw("COLZ")
canvas.Draw()

In [10]:
%jsroot on
hRMS = ROOT.TH1F("RMS cluster","RMS Cluster",22*5,-0.1,2.1)
hRMS.GetXaxis().SetTitle("RMS(cluster distance in track) [mm]")
hRMS.GetYaxis().SetTitle("Count")
for event in verySelectedEvents:
    for track in event.tracks:
        if track.nClusters == 3:
            hRMS.Fill(track.rms)
canvas = ROOT.TCanvas()
canvas.SetGridx(2)
hRMS.Draw("")
canvas.Draw()


In [11]:
#%%capture cap --no-stderr
def evalDisplacement(displacement):
    dcaVals = []
    trackRMSVals = []
    tempEvents=copy.deepcopy(verySelectedEvents) #problem is that the events are changed in the algorithm so a copy must be used
    for event in tempEvents:#loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents.pkl"):
        TrackLeft = Track()
        TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)) for alpide in ["ALPIDE_3","ALPIDE_4"]])
        TrackRight = Track()
        TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
        vertex = Vertex()
        vertex.fromTracks([TrackLeft,TrackRight])
        dcaVals.append(vertex.dca)
        for track in event.tracks:
            if track.nClusters == 3:
                trackRMS = track.rms
                trackRMSVals.append(trackRMS)
    medianDCA = np.median(dcaVals)
    medianTrackRMS = np.median(trackRMSVals)
    
    
    print("Evaluated displacement" ,displacement ,". Got median DCA ",str(medianDCA)[:10],". Got median track RMS ",str(medianTrackRMS)[:10], " and mean DCA: ", str(np.mean(dcaVals))[:5], " and DCA sigma: ", str(np.std(dcaVals))[:5])
    return [medianDCA, medianTrackRMS]


ALPIDEs = ["ALPIDE_0","ALPIDE_1", "ALPIDE_2", "ALPIDE_3", "ALPIDE_4"] 
def findBest(ranges,step):
    print("Round",step)
    uniqueCornArrs = []
    for [x1,x2,y1,y2] in ranges:
        corners = [[x,y,0] for x in [x1,x2] for y in [y1,y2]] 
        uniqueCorners = set(tuple(row) for row in corners)
        uniqueCornArr = [list(row) for row in uniqueCorners]
        uniqueCornArrs.append(uniqueCornArr)
    
    print(uniqueCornArrs)  #prints a list of 5 (winner 0 to 4 displacement sets)(0 to 4 -1 corner alpides)(displacement 0 to 2) displacements (corner neglected )
    displacements = [{
        "ALPIDE_0" : corner0,
        "ALPIDE_1" : corner1,
        "ALPIDE_2" : corner2,
        "ALPIDE_3" : corner3,
        "ALPIDE_4" : corner4, 
    } for corner0 in uniqueCornArrs[0] for corner1 in uniqueCornArrs[1] for corner2 in uniqueCornArrs[2] for corner3 in uniqueCornArrs[3] for corner4 in uniqueCornArrs[4]]
    
    print("Going to try",len(displacements),"displacement configurations.")
    
    
    results = [evalDisplacement(displacement)[0] for displacement in displacements] # list of all the DCA values
    
    winner = results.index(min(results)) #lowest index of the lowest median dca Value. returns a large number since the list is 1024 long I think. 
    
    winningDisplacement = results[winner]
    
    print("Winner of round",step,"is",[displacements[winner][ALPIDEs[i]] for i in range(5)],"with",str(results[winner])[:10]) #switched [i] with [winner] since index out of bound
    
    if step <= 0 :
        return [results[winner],[displacements[winner][ALPIDEs[i]] for i in range(5)]]
    else :
        findBest([[displacements[winner][ALPIDEs[i]][0] ,(x1+x2)/2,displacements[winner][ALPIDEs[i]][1],(y1+y2)/2] for i in range(5)], step-1)
        

    #with open('/home/cehrich/Software/AlignmentOutput/Alignment_200MeV_100eMoreStats.txt', 'w') as f:
        #f.write(str(cap))
    
    #print("Winner of round",step,"is",[displacements[i][winner] for i in range(5)],"with",str(results[winner])[:5]) #switched uniqueCornArrs[i][winner][1] for displacements [i][winner][1] here since index out of bound
    
    #if step <= 0 :
    #    return [results[winner],[displacements[i][winner] for i in range(5)]]
    #else :
    #    findBest([[displacements[i][winner][0] ,(x1+x2)/2,displacements[i][winner][1],(y1+y2)/2] for i in range(5)], step-1)
    
    
    #print("Winner of round",step,"is",[uniqueCornArrs[i][winner] for i in range(5)],"with",str(results[winner])[:5])
    
    #if step <= 0 :
    #    return [results[winner],[uniqueCornArrs[i][winner] for i in range(5)]]
    #else :
    #    findBest([[uniqueCornArrs[i][winner][0] ,(x1+x2)/2,uniqueCornArrs[i][winner][1],(y1+y2)/2] for i in range(5)], step-1)
        

In [ ]:
findBest([[0.2,0.5,-0.2,0.1],[0.4,0.65,-0.2,0.1],[0.55,0.75,-0.25,0],[0.2,0.5,-0.1,0.1],[0.55,0.85,-0.2,0.1]],6)

%%capture cap --no-stderr


with open('/home/cehrich/Software/AlignmentOutput/Alignment_200MeV_100eMoreStatsMoreRefined_Without_Mult_Scattering.txt', 'w') as f:
        f.write(str(cap))

In [ ]:
#from IPython.utils.io import Tee
#with open('/home/cehrich/Software/AlignmentOutput/Alignment_200MeV_100eMoreStatsMoreRefined.txt', 'a') as f, Tee(f, channel='stdout'):
#    output = findBest([[0,0.6,-0.5,0],[0,0.6,-0.5,0],[0,0.6,-0.5,0],[0,0.5,-0.5,0],[0,0.7,-0.5,0]],6)
#    print(output)

In [ ]:
#from IPython.utils.io import Tee
#from io import StringIO
#import sys
#
## Specify the file path for saving the output
#output_file = '/home/cehrich/Software/AlignmentOutput/Alignment_200MeV_100eMoreStatsMoreRefined.txt'
#
## Create a StringIO object to capture the printed output
#output_buffer = StringIO()
#
## Open the file in append mode and redirect the stdout to both the file and the StringIO object
#with open(output_file, 'a') as f, Tee(f, channel='stdout'), Tee(output_buffer, channel='stdout'):
#    # Redirect the stderr to the StringIO object to avoid capturing it in the file
#    sys.stderr = output_buffer
#
#    # Call the findBest function and print its output
#    findBest([[0.2,0.5,-0.2,0.1],[0.4,0.65,-0.2,0.1],[0.55,0.75,-0.25,0],[0.2,0.5,-0.1,0.1],[0.55,0.85,-0.2,0.1]],6)
#
#
## Retrieve the output from the StringIO object and print it in the notebook
#output_buffer.seek(0)
#output = output_buffer.read()
#print(output)

In [28]:
#preAlign


best_displacement_no_multiple_scat ={
    'ALPIDE_0': [0.5, -0.0546875, 0], 
    'ALPIDE_1': [0.65, -0.0546875, 0], 
    'ALPIDE_2': [0.75, -0.06875, 0], 
    'ALPIDE_3': [0.2, -0.125, 0], 
    'ALPIDE_4': [0.5546875, -0.05, 0]
} #Got median DCA  0.05772230 . Got median track RMS  0.48295208  and mean DCA:  0.082  and DCA sigma:  0.130
        

prealigned_events= []
for event in verySelectedEvents:
    for cluster in event.clusters:
        cluster.flipYaxis()
        cluster.alignLocal(best_displacement_no_multiple_scat.get(cluster.detector)) #displacements 6 has no displacement
    prealigned_events.append(event)

In [34]:
f = open('twistAlignmentrefined.out', 'w') 
 
print('this goes to the screen, or Jupyter notebook') 
 
print('Begin file', file=f) 
 


def evalDisplacementTwist(displacement):
    dcaVals = []
    trackRMSVals = []
    tempEvents=copy.deepcopy(prealigned_events) #problem is that the events are changed in the algorithm so a copy must be used
    for event in tempEvents:#loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents.pkl"):
        TrackLeft = Track()
        TrackLeft.fromClusters([event.selectDetector(alpide)[0].twistTopDown(displacement.get(alpide)) for alpide in ["ALPIDE_3","ALPIDE_4"]])
        TrackRight = Track()
        TrackRight.fromClusters([event.selectDetector(alpide)[0].twistTopDown(displacement.get(alpide)) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
        vertex = Vertex()
        vertex.fromTracks([TrackLeft,TrackRight])
        dcaVals.append(vertex.dca)
        for track in event.tracks:
            if track.nClusters == 3:
                trackRMS = track.rms
                trackRMSVals.append(trackRMS)
    medianDCA = np.median(dcaVals)
    medianTrackRMS = np.median(trackRMSVals)
    
    
    print("Evaluated displacement" ,displacement ,". Got median DCA ",str(medianDCA)[:10],". Got median track RMS ",str(medianTrackRMS)[:10], " and mean DCA: ", str(np.mean(dcaVals))[:5], " and DCA sigma: ", str(np.std(dcaVals))[:5], file=f)
    return [medianDCA, medianTrackRMS]


ALPIDEs = ["ALPIDE_0","ALPIDE_1", "ALPIDE_2", "ALPIDE_3", "ALPIDE_4"] 
def findBestTwist(ranges,step):
    print("Round",step)
    uniqueAngArrs = []
    for [alpha1,alpha2] in ranges:
        angles = [alpha for alpha in [alpha1, alpha2]] 
        #uniqueAngles = set(tuple(row) for row in angles)
        #uniqueAngArr = [list(row) for row in uniqueAngles]
        uniqueAngArrs.append(angles) #uniqueAngArr)
    
    print(uniqueAngArrs)  #prints a list of 5 (winner 0 to 4 displacement sets)(0 to 4 -1 angle alpides)(displacement 0 to 2) displacements (angle neglected )
    #print(angles)
    displacements = [{
        "ALPIDE_0" : angle0,
        "ALPIDE_1" : angle1,
        "ALPIDE_2" : angle2,
        "ALPIDE_3" : angle3,
        "ALPIDE_4" : angle4, 
    } for angle0 in uniqueAngArrs[0] for angle1 in uniqueAngArrs[1] for angle2 in uniqueAngArrs[2] for angle3 in uniqueAngArrs[3] for angle4 in uniqueAngArrs[4]]  #for angle0 in angles[0] for angle1 in angles[1] for angle2 in angles[2] for angle3 in angles[3] for angle4 in angles[4]]
     
    print("Going to try",len(displacements),"displacement configurations.")
    
    
    #results = [evalDisplacementTwist(displacement)[0] for displacement in displacements] # list of all the DCA values
    results= None    
   
    if __name__ == "__main__":
        with Pool() as pool:
              results = pool.map(evalDisplacementTwist, displacements)
        print("Program finished!")
    
    
    winner = results.index(min(results)) #lowest index of the lowest median dca Value. returns a large number since the list is 1024 long I think. 
    
    winningDisplacement = results[winner]
    
    print("Winner of round",step,"is",[displacements[winner][ALPIDEs[i]] for i in range(5)],"with",str(results[winner])) #switched [i] with [winner] since index out of bound
    print("Winner of round",step,"is",[displacements[winner][ALPIDEs[i]] for i in range(5)],"with",str(results[winner]), file=f) #switched [i] with [winner] since index out of bound

    if step <= 0 :
        return [results[winner],[displacements[winner][ALPIDEs[i]] for i in range(5)]]
        f.close()
    else :
        findBestTwist([[displacements[winner][ALPIDEs[i]] ,(alpha1+alpha2)/2] for i in range(5)], step-1)
               

this goes to the screen, or Jupyter notebook


In [ ]:
findBestTwist([[0,0.5],[0.1,0.6],[0,0.5],[-0.3,0.3],[0,0.5]],20)

Round 20
[[0, 0.25, 0.5], [0.1, 0.35, 0.6], [0, 0.25, 0.5], [-0.3, 0.0, 0.3], [0, 0.25, 0.5]]
Going to try 243 displacement configurations.
Program finished!
Winner of round 20 is [0.25, 0.6, 0, -0.3, 0] with [0.06295753354266209, 0.48295208736402606]
Round 19
[[0.25, 0.25, 0.25], [0.6, 0.425, 0.25], [0, 0.125, 0.25], [-0.3, -0.024999999999999994, 0.25], [0, 0.125, 0.25]]
Going to try 243 displacement configurations.
Program finished!
Winner of round 19 is [0.25, 0.25, 0.125, -0.024999999999999994, 0.125] with [0.06243050190763637, 0.48295208736402606]
Round 18
[[0.25, 0.1875, 0.125], [0.25, 0.1875, 0.125], [0.125, 0.125, 0.125], [-0.024999999999999994, 0.05, 0.125], [0.125, 0.125, 0.125]]
Going to try 243 displacement configurations.
Program finished!
Winner of round 18 is [0.25, 0.25, 0.125, -0.024999999999999994, 0.125] with [0.06243050190763637, 0.48295208736402606]
Round 17
[[0.25, 0.1875, 0.125], [0.25, 0.1875, 0.125], [0.125, 0.125, 0.125], [-0.024999999999999994, 0.05, 0.125], 

## Explore 15D Space

In [11]:
f = open('/home/cehrich/Software/AlignmentOutput/15DRound4alignment35.out', 'w') 
 
print('this goes to the screen, or Jupyter notebook') 
 
print('this goes to the file', file=f) 
 
      
        
def evalDisplacement15D(displacement):
    dcaVals = []
    trackRMSVals = []
    tempEvents=copy.deepcopy(verySelectedEvents) #problem is that the events are changed in the algorithm so a copy must be used
    for event in tempEvents:#loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents.pkl"):
        TrackLeft = Track()
        TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:2] ) for alpide in ["ALPIDE_3","ALPIDE_4"]]) #corners
        TrackLeft.fromClusters([event.selectDetector(alpide)[0].twistTopDown(displacement.get(alpide)[3]) for alpide in ["ALPIDE_3","ALPIDE_4"]])  #angles
        
        TrackRight = Track()
        TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:2]) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]]) #corners
        TrackRight.fromClusters([event.selectDetector(alpide)[0].twistTopDown(displacement.get(alpide)[3]) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]]) #angles
        
        vertex = Vertex()
        vertex.fromTracks([TrackLeft,TrackRight])
        dcaVals.append(vertex.dca)
        for track in event.tracks:
            if track.nClusters == 3:
                trackRMS = track.rms
                trackRMSVals.append(trackRMS)
                
    DCAnoTail = [dca <= np.median(dcaVals)+ np.std(dcaVals) for dca in dcaVals]
    medianDCA = np.median(dcaVals)
    medianTrackRMS = np.median(trackRMSVals)
    quality = medianDCA + medianTrackRMS/2
    
    print("Evaluated displacement" ,displacement ,". Got median DCA ",str(medianDCA)[:10],". Got median track RMS ",str(medianTrackRMS)[:10], " and mean DCA: ", str(np.mean(dcaVals))[:5], " and DCA sigma x,y,z: ", str(np.std(dcaVals[0]))[:5],", ", str(np.std(dcaVals[1]))[:5],", ", str(np.std(dcaVals[2]))[:5], file=f)

    return [medianDCA, medianTrackRMS, quality]


    
def tryDisplacements(displacementsArgs):
    #result.append(evalDisplacement15D(displacement)[0] for displacement in displacementsArgs)
    
    theseResults=[]
    result="" 
    for displacement in displacementsArgs:
        result=evalDisplacement15D(displacement)[2] # provides the weighted value of the fit
    theseResults.append(result)    
    return theseResults

ALPIDEs = ["ALPIDE_0","ALPIDE_1", "ALPIDE_2", "ALPIDE_3", "ALPIDE_4"] 

def findBest15D(ranges,step):
    print("Round",step)
    uniqueCornArrs = []
    for [x1,x2,y1,y2,alpha1,alpha2] in ranges:
        corners = [[x,y,0,alpha] for x in [x1,x2] for y in [y1,y2] for alpha in [alpha1, alpha2]]
        #angles = [alpha for alpha in [alpha1,alpha2]] 
        
        uniqueCorners = set(tuple(row) for row in corners)
        uniqueCornArr = [list(row) for row in uniqueCorners]
        uniqueCornArrs.append(uniqueCornArr)
        
       # uniqueAngArrs.append(angles)
        
    print(uniqueCornArrs)  #prints a list of 5 (winner 0 to 4 displacement sets)(0 to 4 -1 corner alpides)(displacement 0 to 2) displacements (corner neglected )
    displacements = [{
        "ALPIDE_0" : corner0,
        "ALPIDE_1" : corner1,
        "ALPIDE_2" : corner2,
        "ALPIDE_3" : corner3,
        "ALPIDE_4" : corner4, 
    } for corner0 in uniqueCornArrs[0] for corner1 in uniqueCornArrs[1] for corner2 in uniqueCornArrs[2] for corner3 in uniqueCornArrs[3] for corner4 in uniqueCornArrs[4]]
    
    print("Going to try",len(displacements),"displacement configurations.")
    print("Going to try",len(displacements),"displacement configurations.", file=f)
    #results = [evalDisplacement15D(displacement)[0] for displacement in displacements] # list of all the DCA values
    results= None    
   
    if __name__ == "__main__":
        with Pool() as pool:
              results = pool.map(evalDisplacement15D, displacements)
        print("Program finished!")
    
    
    winner = results.index(min(results)) #lowest index of the lowest median dca Value. returns a large number since the list is 1024 long I think. 
    
    winningDisplacement = results[winner]
    
    print("Winner of round",step,"is",[displacements[winner][ALPIDEs[i]] for i in range(5)],"with",str(results[winner])) #switched [i] with [winner] since index out of bound
    print("Winner of round",step,"is",[displacements[winner][ALPIDEs[i]] for i in range(5)],"with",str(results[winner]), file=f) 
    if step <= 0 :
        return [results[winner],[displacements[winner][ALPIDEs[i]] for i in range(5)]]
        f.close()    
    else :
        findBest15D([[displacements[winner][ALPIDEs[i]][0] ,(x1+x2)/2,displacements[winner][ALPIDEs[i]][1],(y1+y2)/2,displacements[winner][ALPIDEs[i]][3],(alpha1+alpha2)/2] for i in range(5)], step-1)
        

 

this goes to the screen, or Jupyter notebook


In [ ]:
findBest15D([[-0.35,0.35,-0.35, 0.35, -0.35, 0.35],[-0.35,0.35,-0.35, 0.35, -0.35, 0.35],[-0.35,0.35,-0.35, 0.35, -0.35, 0.35],[-0.35,0.35,-0.35, 0.35, -0.35, 0.35],[-0.35,0.35,-0.35, 0.35, -0.35, 0.35]],7)

Round 7
[[[0.35, -0.35, 0, -0.35], [0.35, -0.35, 0, 0.35], [0.35, 0.35, 0, -0.35], [-0.35, -0.35, 0, -0.35], [-0.35, 0.35, 0, -0.35], [0.35, 0.35, 0, 0.35], [-0.35, -0.35, 0, 0.35], [-0.35, 0.35, 0, 0.35]], [[0.35, -0.35, 0, -0.35], [0.35, -0.35, 0, 0.35], [0.35, 0.35, 0, -0.35], [-0.35, -0.35, 0, -0.35], [-0.35, 0.35, 0, -0.35], [0.35, 0.35, 0, 0.35], [-0.35, -0.35, 0, 0.35], [-0.35, 0.35, 0, 0.35]], [[0.35, -0.35, 0, -0.35], [0.35, -0.35, 0, 0.35], [0.35, 0.35, 0, -0.35], [-0.35, -0.35, 0, -0.35], [-0.35, 0.35, 0, -0.35], [0.35, 0.35, 0, 0.35], [-0.35, -0.35, 0, 0.35], [-0.35, 0.35, 0, 0.35]], [[0.35, -0.35, 0, -0.35], [0.35, -0.35, 0, 0.35], [0.35, 0.35, 0, -0.35], [-0.35, -0.35, 0, -0.35], [-0.35, 0.35, 0, -0.35], [0.35, 0.35, 0, 0.35], [-0.35, -0.35, 0, 0.35], [-0.35, 0.35, 0, 0.35]], [[0.35, -0.35, 0, -0.35], [0.35, -0.35, 0, 0.35], [0.35, 0.35, 0, -0.35], [-0.35, -0.35, 0, -0.35], [-0.35, 0.35, 0, -0.35], [0.35, 0.35, 0, 0.35], [-0.35, -0.35, 0, 0.35], [-0.35, 0.35, 0, 0.35]]]
G